# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [1]:
import glob

import numpy as np
import xarray as xr

from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

SET_NAME = "enso_diags"
SET_DIR = "861-time-series-multiple"

DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{SET_DIR}/{SET_NAME}/**"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/{SET_NAME}/**"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [2]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_count = 0

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(SET_DIR, "enso-main")

        if fp_dev not in MAIN_GLOB:
            print(f"No production file found to compare with {fp_dev}!")
            missing_count += 1

    for fp_dev in DEV_GLOB:
        fp_main = fp_main.replace("enso-main", SET_DIR)

        if fp_main not in DEV_GLOB:
            print(f"No development file found to compare with {fp_main}!")
            missing_count += 1

    print(f"Number of files missing: {missing_count}")

In [3]:
def _get_relative_diffs() -> int:
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    mismatches = []

    for fp_main in MAIN_GLOB:
        if "test.nc" in fp_main or "ref.nc" in fp_main:
            fp_dev = fp_main.replace("enso-main", SET_DIR)

            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")

            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)

            if "regression" in fp_main:
                var_key = fp_main.split("/")[-1].split("-")[2].upper()
                var_key_cdat = f"{var_key}-regression-over-nino"
            elif "feedback" in fp_main:
                var_key = fp_main.split("/")[-1].split("-")[1].upper()
                var_key_cdat = f"{var_key}-feedback"

            print(f"    * var_key: {var_key}")

            dev_data = ds1[var_key].values
            # main_data = ds2[var_key_cdat].values[1:-1]
            main_data = ds2[var_key_cdat]

            if dev_data is None or main_data is None:
                print("    * Could not find variable key in the dataset(s)")
                continue

            try:
                np.testing.assert_allclose(
                    dev_data,
                    main_data,
                    atol=atol,
                    rtol=rtol,
                )
            except (KeyError, AssertionError) as e:
                if "mismatch" in str(e):
                    np.testing.assert_allclose(
                        dev_data,
                        main_data[1:-1],
                        atol=atol,
                        rtol=rtol,
                    )
                else:
                    print(f"    {e}")
                    mismatches.append((fp_dev, fp_main))
            else:
                print(f"    * All close and within relative tolerance ({rtol})")

    return mismatches

## 1. Check for matching and equal number of files


In [4]:
_check_if_files_found()

In [5]:
DEV_GLOB

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/FSNS-feedback/feedback-FSNS-NINO3-TS-NINO3_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/FSNS-feedback/feedback-FSNS-NINO3-TS-NINO3_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/LHFLX-feedback/feedback-LHFLX-NINO3-TS-NINO3_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/LHFLX-feedback/feedback-LHFLX-NINO3-TS-NINO3_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_diff.nc',
 '/global/cfs/cdirs/

In [6]:
MAIN_GLOB

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/FSNS-feedback/feedback-FSNS-NINO3-TS-NINO3_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/FSNS-feedback/feedback-FSNS-NINO3-TS-NINO3_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/LHFLX-feedback/feedback-LHFLX-NINO3-TS-NINO3_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/LHFLX-feedback/feedback-LHFLX-NINO3-TS-NINO3_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_diff.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34

In [7]:
_check_if_missing_files()

Number of files missing: 0


In [8]:
_check_if_matching_filecount()

Matching file count (30 and 30).


## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [9]:
mismatches = _get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_ref.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_ref.nc
    * var_key: FLNS
    
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 120 / 120 (100%)
Max absolute difference: 0.08201782
Max relative difference: 0.64689754
 x: array([ 4.58544 ,  2.485706,  3.111711,  0.767667,  1.648402,  3.223995,
        1.59968 ,  0.373345,  1.550645, -0.038671, -1.088026,  0.199513,
       -1.694707,  0.574215, -1.211548, -0.088548,  1.481813,  0.685491,...
 y: array([ 4.576222,  2.494163,  3.120666,  0.74798 ,  1.660498,  3.238505,
        1.615224,  0.385147,  1.570752, -0.061149, -1.084575,  0.19586 ,
       -1.688319,  0.608236, -1.16629 , -0.060196,  1.414754,  0.636474,...
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_di

## Results

- For the CDAT regression netCDF files, we remove the extra start and end lat coordinates that are added via the `"ccb"` slice flag. This ensures arrays and results are aligned.
- For the feedback netCDF files, the two extra points results in what seems like a large differences in the anomalies.


In [10]:
mismatches

[('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_ref.nc',
  '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/FLNS-feedback/feedback-FLNS-NINO3-TS-NINO3_ref.nc'),
 ('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/FSNS-feedback/feedback-FSNS-NINO3-TS-NINO3_ref.nc',
  '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/FSNS-feedback/feedback-FSNS-NINO3-TS-NINO3_ref.nc'),
 ('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/LHFLX-feedback/feedback-LHFLX-NINO3-TS-NINO3_ref.nc',
  '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-main/enso_diags/LHFLX-feedback/feedback-LHFLX-NINO3-TS-NINO3_ref.nc'),
 ('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/enso_diags/NET_FLUX_SRF-feedback/feedback-NET_FLUX_SRF-NINO3-TS-NINO3_ref.nc',
  '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/enso-m

In [11]:
ds1 = xr.open_dataset(mismatches[0][0])

In [12]:
ds2 = xr.open_dataset(mismatches[0][1])

In [13]:
ds1

<xarray.Dataset> Size: 2kB
Dimensions:  (time: 120)
Coordinates:
  * time     (time) datetime64[ns] 960B 2001-01-16T12:00:00 ... 2010-12-16T12...
Data variables:
    FLNS     (time) float64 960B ...

In [14]:
ds2

<xarray.Dataset> Size: 4kB
Dimensions:        (time: 120, bound: 2)
Coordinates:
  * time           (time) datetime64[ns] 960B 2001-01-16T12:00:00 ... 2010-12...
Dimensions without coordinates: bound
Data variables:
    bounds_time    (time, bound) datetime64[ns] 2kB ...
    FLNS-feedback  (time) float64 960B ...
Attributes:
    Conventions:  CF-1.0

In [15]:
print(ds1["FLNS"].sum().item(), ds2["FLNS-feedback"].sum().item())
print(ds1["FLNS"].mean().item(), ds2["FLNS-feedback"].mean().item())
print(ds1["FLNS"].min().item(), ds2["FLNS-feedback"].min().item())
print(ds1["FLNS"].max().item(), ds2["FLNS-feedback"].max().item())
print(ds1["FLNS"].std().item(), ds2["FLNS-feedback"].std().item())

-0.2028542676624383 -0.202113868730045
-0.0016904522305203193 -0.0016842822394170416
-8.156214274999115 -8.103394765085596
4.585439916659517 4.5762217718118094
2.281253170375751 2.2717773078329726
